<a href="https://colab.research.google.com/github/DerekChuang/tensorflow_practice/blob/master/Dataset_API_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![替代文字](https://pic2.zhimg.com/80/v2-f9f42cc5c00573f7baaa815795f1ce45_hd.jpg
)

Dataset可以看作是相同类型“元素”的有序列表

先以最简单的，Dataset的每一个元素是一个数字为例：

In [0]:

import tensorflow as tf
import numpy as np

dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))


如何将这个dataset中的元素取出呢？方法是从Dataset中示例化一个Iterator，然后对Iterator进行迭代。

In [3]:
#在非Eager模式下，读取上述dataset中元素的方法为：

iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(5):
        print(sess.run(one_element))

1.0
2.0
3.0
4.0
5.0


语句iterator = dataset.make_one_shot_iterator()从dataset中实例化了一个Iterator，这个Iterator是一个“one shot iterator”，即只能从头到尾读取一次。one_element = iterator.get_next()表示从iterator里取出一个元素。由于这是非Eager模式，所以one_element只是一个Tensor，并不是一个实际的值。调用sess.run(one_element)后，才能真正地取出一个值。

如果一个dataset中元素被读取完了，再尝试sess.run(one_element)的话，就会抛出tf.errors.OutOfRangeError异常，这个行为与使用队列方式读取数据的行为是一致的。在实际程序中，可以在外界捕捉这个异常以判断数据是否读取完，请参考下面的代码：

In [4]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    try:
        while True:
            print(sess.run(one_element))
    except tf.errors.OutOfRangeError:
        print("end!")

1.0
2.0
3.0
4.0
5.0
end!


在Eager模式中，创建Iterator的方式有所不同。是通过tfe.Iterator(dataset)的形式直接创建Iterator并迭代。迭代时可以直接取出值，不需要使用sess.run()：


# 从内存中创建更复杂的Dataset


其实，tf.data.Dataset.from_tensor_slices的功能不止如此，它的真正作用是切分传入Tensor的第一个维度，生成相应的dataset。

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(np.random.uniform(size=(5, 2)))


传入的数值是一个矩阵，它的形状为(5, 2)，tf.data.Dataset.from_tensor_slices就会切分它形状上的第一个维度，最后生成的dataset中一个含有5个元素，每个元素的形状是(2, )，即每个元素是矩阵的一row



我们可能还希望Dataset中的每个元素具有更复杂的形式，如每个元素是一个Python中的元组，或是Python中的词典。例如，在图像识别问题中，一个元素可以是{"image": image_tensor, "label": label_tensor}的形式，这样处理起来更方便。

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "a": np.array([1.0, 2.0, 3.0, 4.0, 5.0]),                                       
        "b": np.random.uniform(size=(5, 2))
    }
)

#最终dataset中的一个元素就是类似于{"a": 1.0, "b": [0.9, 0.1]}

利用tf.data.Dataset.from_tensor_slices创建每个元素是一个tuple的dataset也是可以的：



In [0]:
dataset = tf.data.Dataset.from_tensor_slices(
  (np.array([1.0, 2.0, 3.0, 4.0, 5.0]), np.random.uniform(size=(5, 2)))
)

# 对Dataset中的元素做变换：Transformation

Dataset支持一类特殊的操作：Transformation。一个Dataset通过Transformation变成一个新的Dataset。通常我们可以通过Transformation完成数据变换，打乱，组成batch，生成epoch等一系列操作。

常用的Transformation有：

map
batch
shuffle
repeat


# （1）map

map接收一个函数，Dataset中的每个元素都会被当作这个函数的输入，并将函数返回值作为新的Dataset，如我们可以对dataset中每个元素的值加1：

In [0]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
dataset = dataset.map(lambda x: x + 1) # 2.0, 3.0, 4.0, 5.0, 6.0

# （2）batch

batch就是将多个元素组合成batch，如下面的程序将dataset中的每个元素组成了大小为32的batch：

In [0]:
dataset = dataset.batch(32)


# （3）shuffle

shuffle的功能为打乱dataset中的元素，它有一个参数buffersize，表示打乱时使用的buffer的大小：

In [0]:
dataset = dataset.shuffle(buffer_size=10000)


# （4）repeat

repeat的功能就是将整个序列重复多次，主要用来处理机器学习中的epoch，假设原先的数据是一个epoch，使用repeat(5)就可以将之变成5个epoch：

In [0]:
dataset = dataset.repeat(5)


如果直接调用repeat()的话，生成的序列就会无限重复下去，没有结束，因此也不会抛出tf.errors.OutOfRangeError异常：

In [0]:
dataset = dataset.repeat()


# 例子：读入磁盘图片与对应label

讲到这里，我们可以来考虑一个简单，但同时也非常常用的例子：读入磁盘中的图片和图片相应的label，并将其打乱，组成batch_size=32的训练样本。在训练时重复10个epoch。

In [17]:
#掛載 google drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
!ls '/content/drive/My Drive/太平山照片'


R0019664.JPG  R0019702.JPG  R0019740.JPG  R0019779.JPG	R0019822.JPG
R0019665.JPG  R0019703.JPG  R0019741.JPG  R0019780.JPG	R0019823.JPG
R0019666.JPG  R0019704.JPG  R0019742.JPG  R0019784.JPG	R0019824.JPG
R0019667.JPG  R0019705.JPG  R0019743.JPG  R0019785.JPG	R0019825.JPG
R0019668.JPG  R0019706.JPG  R0019744.JPG  R0019786.JPG	R0019826.JPG
R0019669.JPG  R0019707.JPG  R0019745.JPG  R0019787.JPG	R0019827.JPG
R0019670.JPG  R0019708.JPG  R0019746.JPG  R0019788.JPG	R0019828.JPG
R0019671.JPG  R0019709.JPG  R0019747.JPG  R0019789.JPG	R0019829.JPG
R0019672.JPG  R0019710.JPG  R0019748.JPG  R0019790.JPG	R0019830.JPG
R0019673.JPG  R0019711.JPG  R0019749.JPG  R0019791.JPG	R0019831.JPG
R0019674.JPG  R0019712.JPG  R0019750.JPG  R0019792.JPG	R0019832.JPG
R0019675.JPG  R0019713.JPG  R0019751.JPG  R0019793.JPG	R0019833.JPG
R0019676.JPG  R0019714.JPG  R0019752.JPG  R0019794.JPG	R0019834.JPG
R0019677.JPG  R0019715.JPG  R0019753.JPG  R0019795.JPG	R0019835.JPG
R0019678.JPG  R0019716.JPG  R0019754.JPG  R00197

In [30]:
! ls '/content/drive' 

'My Drive'


In [32]:
# 函数的功能时将filename对应的图片文件读进来，并缩放到统一的大小
def _parse_function(filename, label):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_image(image_string)
  image_resized = tf.image.resize_images(image_decoded, [28, 28])
  return image_resized, label

# 图片文件的列表
filenames = tf.constant(["/content/drive/My Drive/tp_photo/R0019664.jpg"])
# label[i]就是图片filenames[i]的label
labels = tf.constant([0])

# 此时dataset中的一个元素是(filename, label)
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# 此时dataset中的一个元素是(image_resized, label)
dataset = dataset.map(_parse_function)

# 此时dataset中的一个元素是(image_resized_batch, label_batch)
dataset = dataset.shuffle(buffersize=1000).batch(32).repeat(10)

ValueError: ignored

https://www.jianshu.com/p/8c168fc0e01e